In [0]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

In [2]:
! curl -O  'https://raw.githubusercontent.com/suyongeum/PMLWS2018_WS4/master/shakespeare.txt'
input_file = 'shakespeare.txt'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1089k  100 1089k    0     0  2453k      0 --:--:-- --:--:-- --:--:-- 2453k


In [3]:
# data I/O
data = open(input_file, 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print ('data has %d characters, %d unique.' % (data_size, vocab_size))

char_to_ix = { ch:i for i,ch in enumerate(chars) } # make a dictionary format. See below how it looks like
ix_to_char = { i:ch for i,ch in enumerate(chars) } 

print (char_to_ix)
print (ix_to_char)

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

data has 1115390 characters, 65 unique.
{'i': 0, 'D': 1, 'r': 2, 't': 3, 'n': 4, 'P': 5, 'O': 6, 'o': 7, 'v': 8, 'J': 9, 'G': 10, '$': 11, 'B': 12, ';': 13, 'V': 14, 'h': 15, 'T': 16, 'l': 17, 'X': 18, '?': 19, 'W': 20, 'H': 21, '\n': 22, 's': 23, 'M': 24, '.': 25, '-': 26, 'Z': 27, 'C': 28, 'S': 29, 'y': 30, 'Y': 31, 'I': 32, ':': 33, 'b': 34, 'x': 35, '3': 36, 'p': 37, 'R': 38, 'f': 39, 'e': 40, 'w': 41, 'E': 42, '&': 43, "'": 44, 'N': 45, 'K': 46, 'z': 47, 'u': 48, 'm': 49, 'U': 50, 'd': 51, ' ': 52, 'q': 53, 'g': 54, 'j': 55, 'c': 56, 'F': 57, ',': 58, 'a': 59, 'k': 60, 'A': 61, 'L': 62, 'Q': 63, '!': 64}
{0: 'i', 1: 'D', 2: 'r', 3: 't', 4: 'n', 5: 'P', 6: 'O', 7: 'o', 8: 'v', 9: 'J', 10: 'G', 11: '$', 12: 'B', 13: ';', 14: 'V', 15: 'h', 16: 'T', 17: 'l', 18: 'X', 19: '?', 20: 'W', 21: 'H', 22: '\n', 23: 's', 24: 'M', 25: '.', 26: '-', 27: 'Z', 28: 'C', 29: 'S', 30: 'y', 31: 'Y', 32: 'I', 33: ':', 34: 'b', 35: 'x', 36: '3', 37: 'p', 38: 'R', 39: 'f', 40: 'e', 41: 'w', 42: 'E', 43: 

In [0]:
def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
    
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  
  # If you want to understand the code below, refer to 
  # http://www.suyongeum.com/ML/lectures/LectureW10_20180621_print.pdf 
  # slides: 11, 12, 13
  for t in reversed(range(len(inputs))): 
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # 
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

In [0]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0

while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 1000 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print ('----------------------------------------------------------------------------------------------\n \\
           %s \n-----------------------------------------------------------------------------------------------' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 1000 == 0: print ('iter %d, loss: %f \n\n' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 


----------------------------------------------------------------------------------------------
 \           xq-O.q:SimAU!rP&or 'FhXXH iFSa!ksgaIeW;f,fLiU'IXZCiynkDKCRwQcby.mSJX$swSi
?fL3.Q,&uTRaIncIRk-iaUglpRlzCvlPcGMjasplvh'3wZG'dZ! dSceiF!uoNv xp&!Wd.Z
Hq$'aZestyf xccy?z-gG.jTeiZs XJb&y3MY:uq'YPjiOsZc!x'm 
-----------------------------------------------------------------------------------------------
iter 0, loss: 104.359684 


----------------------------------------------------------------------------------------------
 \            tn thaB pairaf nachedi,,rsanasd hiur,l -outy
Uen land ahnd  twath.
Tf periag' 'cs, fame tgoThety aco''s psoy coI fs thour.
Tee
:
JecgomThtomd 'r apsacaulvenl:thod', ms th! be wabrci?e,
Fo Rhimro Teer 
-----------------------------------------------------------------------------------------------
iter 1000, loss: 86.674806 


----------------------------------------------------------------------------------------------
 \           wfantrd it,

FanBa or,